In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv('Verkada_Data.csv', sep=';')
print(data)
print(data.shape)
print(data.describe())

     client_status       source mql_created_date
0              SQL        Email         4/1/2019
1              SQL        Email         4/1/2019
2              SQL        Email         4/1/2019
3              SQL        Email         4/1/2019
4              SQL        Email         4/1/2019
...            ...          ...              ...
2154           SQL  Paid Social        4/27/2019
2155           MQL  Paid Social        4/27/2019
2156           SQL  Paid Social        4/27/2019
2157           SQL  Paid Social        4/27/2019
2158           MQL  Paid Social        4/27/2019

[2159 rows x 3 columns]
(2159, 3)
       client_status       source mql_created_date
count           2159         2159             2159
unique             2            4               27
top              MQL  Paid Social        4/26/2019
freq            1318         1512              473


In [3]:
#Rearranging date

#Date

def modify_date(original_date):
    if len(original_date.split('/')[1])==1:
        return '0'+original_date.split('/')[1]+'/04/2019'
    else:
        return original_date.split('/')[1]+'/04/2019'


data['mql_created_date_mod'] = data['mql_created_date'].apply(lambda x: modify_date(x))

data['mql_created_date_mod'] = pd.to_datetime(data['mql_created_date_mod'], format='%d/%m/%Y')

# Name

data['source_renamed'] = pd.Series(np.where(data['source']=='Email','01. Email',\
          np.where(data['source']=='Direct Mail','02. Direct Mail',\
          np.where(data['source']=='Paid Social','03. Paid Social','04. Other'))))

# week_of_month

conditions_week_of_month = [
    data['mql_created_date_mod']<='2019-04-07',
    (data['mql_created_date_mod']>'2019-04-07') & (data['mql_created_date_mod']<='2019-04-14'),
    (data['mql_created_date_mod']>'2019-04-14') & (data['mql_created_date_mod']<='2019-04-21'),
    data['mql_created_date_mod']>'2019-04-21'
]
choices_week_of_month = ['01. First Week','02. Second Week','03. Third Week','04. Fourth Week']

data['week_of_month'] = pd.Series(np.select(conditions_week_of_month, choices_week_of_month))


# flg_sql

data['flg_sql'] = pd.Series(np.where(data['client_status']=='SQL',1,0))

# flg_mql_ever

data['flg_mql_ever'] = 1

print(data)


     client_status       source mql_created_date mql_created_date_mod  \
0              SQL        Email         4/1/2019           2019-04-01   
1              SQL        Email         4/1/2019           2019-04-01   
2              SQL        Email         4/1/2019           2019-04-01   
3              SQL        Email         4/1/2019           2019-04-01   
4              SQL        Email         4/1/2019           2019-04-01   
...            ...          ...              ...                  ...   
2154           SQL  Paid Social        4/27/2019           2019-04-27   
2155           MQL  Paid Social        4/27/2019           2019-04-27   
2156           SQL  Paid Social        4/27/2019           2019-04-27   
2157           SQL  Paid Social        4/27/2019           2019-04-27   
2158           MQL  Paid Social        4/27/2019           2019-04-27   

       source_renamed    week_of_month  flg_sql  flg_mql_ever  
0           01. Email   01. First Week        1            

In [65]:
print(pd.concat([data['client_status'].value_counts(normalize=True),data['client_status'].value_counts()], axis=1))
print(pd.concat([data['source'].value_counts(normalize=True),data['source'].value_counts()], axis=1))
print(pd.concat([data['source_renamed'].value_counts(normalize=True),data['source_renamed'].value_counts()], axis=1))
print(pd.concat([data['mql_created_date_mod'].value_counts(normalize=True),data['mql_created_date_mod'].value_counts()], axis=1).reset_index().rename(columns={'index': 'date'}).sort_values('date').reset_index(drop=True))
print(pd.concat([data['flg_sql'].value_counts(normalize=True),data['flg_sql'].value_counts()], axis=1))
print(pd.concat([data['flg_mql_ever'].value_counts(normalize=True),data['flg_mql_ever'].value_counts()], axis=1))
print(pd.concat([data['week_of_month'].value_counts(normalize=True),data['week_of_month'].value_counts()], axis=1).reset_index().rename(columns={'index': 'week'}).sort_values('week').reset_index(drop=True))

     client_status  client_status
MQL       0.610468           1318
SQL       0.389532            841
               source  source
Paid Social  0.700324    1512
Direct Mail  0.154701     334
Email        0.135711     293
Other        0.009264      20
                 source_renamed  source_renamed
03. Paid Social        0.700324            1512
02. Direct Mail        0.154701             334
01. Email              0.135711             293
04. Other              0.009264              20
         date  mql_created_date_mod  mql_created_date_mod
0  2019-04-01              0.008800                    19
1  2019-04-02              0.003242                     7
2  2019-04-03              0.022233                    48
3  2019-04-04              0.003242                     7
4  2019-04-05              0.002316                     5
5  2019-04-06              0.003705                     8
6  2019-04-07              0.000926                     2
7  2019-04-08              0.001853         

In [5]:
#print(data.groupby('week_of_month').agg({'flg_sql': 'sum', 'flg_mql_ever': 'sum','week_of_month': 'count'}))

#Recreating tables in Python
print( data.groupby('week_of_month').agg({
    'flg_sql': 'sum',
    'flg_mql_ever': 'sum',
}).assign(
    conversion_rate=lambda x: x['flg_sql'] / x['flg_mql_ever']
).reset_index().sort_values('week_of_month').reset_index(drop=True))

print( data.groupby('source_renamed').agg({
    'flg_sql': 'sum',
    'flg_mql_ever': 'sum',
}).assign(
    conversion_rate=lambda x: x['flg_sql'] / x['flg_mql_ever']
).reset_index().sort_values('source_renamed').reset_index(drop=True))


result1=data.pivot_table(values='flg_mql_ever',index='source_renamed', columns='week_of_month', aggfunc='sum', fill_value=0)
result2=data.pivot_table(values='flg_sql',index='source_renamed', columns='week_of_month', aggfunc='sum', fill_value=0)
result3=data.pivot_table(values='flg_mql_ever',index='source_renamed', columns='week_of_month', aggfunc='sum', fill_value=0).apply(lambda x: x*100/sum(x))


print(result2/result1)
print(result1)
print(result3)


     week_of_month  flg_sql  flg_mql_ever  conversion_rate
0   01. First Week       45            96         0.468750
1  02. Second Week       64           136         0.470588
2   03. Third Week      137           408         0.335784
3  04. Fourth Week      595          1519         0.391705
    source_renamed  flg_sql  flg_mql_ever  conversion_rate
0        01. Email      243           293         0.829352
1  02. Direct Mail      126           334         0.377246
2  03. Paid Social      463          1512         0.306217
3        04. Other        9            20         0.450000
week_of_month    01. First Week  02. Second Week  03. Third Week  \
source_renamed                                                     
01. Email              0.697674         0.716418        0.836735   
02. Direct Mail        0.181818         0.263158        0.349515   
03. Paid Social        0.181818         0.220000        0.234375   
04. Other              0.450000              NaN             NaN   

w